In [42]:
%pip install PyMuPDF

In [57]:
%pip install flask
%pip install --upgrade watchdog

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/83.0 kB ? eta -:--:--
   ---------------------------------------- 83.0/83.0 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: watchdog
    Found existing installation: watchdog 2.1.6
    Uninstalling watchdog-2.1.6:
      Successfully uninstalled watchdog-2.1.6
Note: you may need to restart the kernel to use updated packages.


In [43]:
from elasticsearch import Elasticsearch
import fitz
import os

In [44]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=["elastic","uxVFMV452qfPJkXeYlVW"],
    ca_certs="C:\\Users\\aonno\\Downloads\\elasticsearch-8.13.4-windows-x86_64\\elasticsearch-8.13.4\\config\\certs\\http_ca.crt"
)

es.ping()

True

In [45]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return "Error getting the file"

In [46]:
# Function to check if a file has already been ingested
def is_file_ingested(file_name, es_client, es_index):
    query = {
        "query": {
            "term": {
                "file_name.keyword": file_name
            }
        }
    }
    res = es_client.search(index=es_index, body=query)
    return len(res['hits']['hits']) > 0

In [47]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [48]:
es_index = "pdf_documents"

In [49]:
# Function to prepare data for Elasticsearch
def prepare_data_for_elasticsearch(pdf_folder):
    data_list = []
    for root, _, files in os.walk(pdf_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                if not is_file_ingested(file, es, es_index):
                    pdf_path = os.path.join(root, file)
                    text_content = extract_text_from_pdf(pdf_path)
                    content_vector = model.encode(text_content)
                    print(content_vector)
                    data = {
                        "file_name": file,
                        "location": pdf_path,
                        "content": text_content,
                        "content_vector": content_vector
                    }
                    data_list.append(data)
                else:
                    print("file already ingested.Skipping")
    return data_list


In [50]:
# Function to index data into Elasticsearch
def index_data_to_elasticsearch(data_list, es_index, es_client):
    for data in data_list:
        es_client.index(index=es_index, body=data)

In [51]:
es.ping()

True

In [52]:
pdf_folder = ".\pdf"  # Replace with the path to your PDF folder
es_host = "http://localhost:9200"  # Replace with your Elasticsearch host
es_index = "pdf_documents"  # Replace with your Elasticsearch index name

# Initialize Elasticsearch client
# es_client = Elasticsearch([es_host])
# es is es client

# Extract data from PDFs and prepare for Elasticsearch
data_list = prepare_data_for_elasticsearch(pdf_folder)

# Index data into Elasticsearch
index_data_to_elasticsearch(data_list, es_index, es)

print(f"Indexed {len(data_list)} documents into Elasticsearch index '{es_index}'.")


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\aonno\AppData\Local\Temp\ipykernel_5208\3510188874.py:1: SyntaxWarning: invalid escape sequence '\p'
  pdf_folder = ".\pdf"  # Replace with the path to your PDF folder


file already ingested.Skipping
[ 3.06778233e-02 -5.00367656e-02 -1.60034169e-02 -5.88151105e-02
 -1.16284378e-02 -2.52136048e-02  2.00418923e-02  9.98345949e-03
 -4.81597446e-02 -6.70200447e-03 -1.28003862e-02 -5.32402704e-03
  5.57958260e-02  1.95952901e-03  5.50487302e-02  2.23394879e-03
  2.03479417e-02 -1.93522219e-02 -6.88359290e-02  4.37771082e-02
 -4.05227207e-02 -9.58875846e-03 -5.55759929e-02 -6.74029929e-04
  6.51760548e-02  2.62193382e-02 -1.79248042e-02 -4.30681035e-02
 -3.16004455e-02 -9.10640433e-02  8.95293579e-02  2.42825858e-02
  5.14826849e-02 -2.18621492e-02  1.85553381e-06  5.65091753e-03
  3.18039693e-02 -7.67268892e-03 -4.82764095e-02  3.28235440e-02
  5.01886457e-02 -4.81482409e-03  2.28731409e-02 -8.30701087e-03
 -2.49514561e-02  8.27300735e-03 -3.13783288e-02  3.19388998e-03
  5.13868332e-02  9.61054713e-02 -1.11739282e-02  1.55436592e-02
 -9.80123058e-02  6.89655822e-03 -5.04011549e-02 -3.35399956e-02
 -4.26507518e-02  3.28134112e-02 -3.38658108e-03  3.1896937

In [53]:
# ### creta new index in elastic search
# from indexMapping import indexMapping
# try:
#     es.indices.create(index="all_products", mappings=indexMapping)
# except Exception as e:
#     print(e)

In [54]:
input_keyword = "Kiesha"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "content_vector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="pdf_documents", knn=query , source=["Name","location", "content"])
res["hits"]["hits"]

C:\Users\aonno\AppData\Local\Temp\ipykernel_5208\2653000530.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="pdf_documents", knn=query , source=["Name","location", "content"])


[{'_index': 'pdf_documents',
  '_id': 'rOh5t48Bs1_zjc2DYD1k',
  '_score': 0.51249766,
  '_ignored': ['content.keyword'],
  '_source': {'location': '.\\pdf\\inec.pdf',
   'content': "Author’s Biography \nAybars Oruc is a Marine Engineer. After he had worked as an engineer on different types of merchant vessels such as \naframax tanker, LPG tanker and container vessel, he worked as HSEQ Coordinator, and then HSEQ Superintendent in a \ntanker management company for 4 years. He is a Ph.D. candidate at the NTNU. His research field is maritime cyber security. \nClaims of State-Sponsored Cyberattack in the Maritime Industry \nA Oruc* M.Sc. MIET MIMarEST \n* Norwegian University of Science and Technology, Norway\n* Corresponding Author. Email: aybars.oruc@ntnu.no\nSynopsis \nDevelopments in technology bring inherent risks along with convenience. Undoubtedly, cyberattacks \nconstitute one potentially serious risk. While a stereotypical scenario involves a curious teenager sitting in \nfront of 